In [64]:
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3
import pandas as pd
import numpy as np
import io
from sagemaker.session import Session
from sagemaker import get_execution_role
sagemaker_session = Session()

feature_group = FeatureGroup(name = 'BtcPriceData')

query = feature_group.athena_query()

table = query.table_name
table
default_s3_bucket_name = 'binance-btc-prices'
n_steps = 10

2024-01-10 00:37:46,696 INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2024-01-10 00:37:46,836 INFO: Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [65]:
q= f'''
SELECT *
FROM
    (SELECT *,
    row_number()
    OVER
        (PARTITION BY "open_time"
        ORDER BY "close_time" DESC, Api_Invocation_Time DESC, write_time DESC)
    AS row_number
    FROM "sagemaker_featurestore"."btcpricedata_1704246894")
WHERE row_number = 1 ORDER BY close_time desc limit {n_steps};
'''

In [66]:
# run Athena query. The output is loaded to a Pandas dataframe.
dataset = pd.DataFrame()
query.run(query_string=q, output_location='s3://'+default_s3_bucket_name+'/query_results/')
query.wait()
dataset = query.as_dataframe()

2024-01-10 00:37:49,214 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:37:54,267 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:37:59,321 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:38:04,380 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:38:09,424 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:38:14,480 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:38:19,526 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:38:24,570 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 is being executed.
2024-01-10 00:38:29,613 INFO: Query 71dca273-c44e-41ac-aaea-5ed45188ddd3 successfully executed.


In [56]:
import hopsworks
import joblib
project = hopsworks.login()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) DD2223_lab1
	 (2) project0



Enter project to access:  1



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/206996


In [60]:
mr = project.get_model_registry()
model = mr.get_model("lstmModel", version=3)
model_dir = model.download()
#model = joblib.load(model_dir + "/lstm_model.h5")

Connected. Call `.close()` to terminate connection gracefully.


In [61]:
import os
os.listdir(model_dir)

['model_schema.json', 'lstm_model.h5']

In [62]:
model_dir

'/tmp/7a941799-82cf-4662-a561-01d7ac828818/lstmModel/3'

In [63]:
from tensorflow.keras.models import load_model
# Load the model
model = load_model(os.path.join(model_dir,'lstm_model.h5' ))#+ '/lstm_model.h5'))
model

In [102]:
initial_sequence = np.array(dataset.close_price)

array([[46920.91, 46944.  , 47060.  , 47099.99, 47031.18, 47040.  ,
        43929.02, 43858.24, 43773.54, 43619.99]])

In [105]:
def predict_hours(initial_sequence, h = 6):
    recursive_preds = []
    initial_sequence = np.array(dataset.close_price)
    for i in range(h * 12):
        input_ = initial_sequence.reshape(1,10)
        pred = model.predict(input_,verbose = 0)[0][0]
        recursive_preds.append(pred)
        initial_sequence = np.append(initial_sequence,pred)[-10:]
    return recursive_preds
predict_hours(initial_sequence)

[27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285,
 27794.285]